In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [207]:
from urllib.request import urlopen as uReq

In [208]:
from bs4 import BeautifulSoup as soup
import requests

In [230]:
## creates a dictionary with all known info from upcitemdb.com

def lookup(query):
    ## append search UPC to the database website
    my_url = "https://www.upcitemdb.com/upc/"+query
    
    #open urllib client, accounting for page not-existing
    try:
        uClient = uReq(my_url)
    except:
        return None
    
    # grab page html, save as soup
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,"html.parser")
    
    # find the table of item details, use first bc only 1
    container = page_soup.findAll("table",{"class":"detail-list"})[0]
    
    ## create blank dictionary
    info_dict={}
    
    ## find product name
    
    # climb into blah is associated with blah
    pieces = str(page_soup.head.findAll("meta")[1].meta).split(",")
    #get rid of first thing
    pieces.pop(0)
    
    #create empty string
    output_str = ""
    # a will serve as a toggle switch to append pieces to output or not
    a=True
    # going through each term, toggle off a once we hit upc lookup database
    for piece in pieces:    
        if piece==" upc lookup database":
            a=False
        if a:
            output_str+=piece
    
    def clean(x):
        x=x.replace("&amp","&")
        for char in x:
            if char.isnumeric():
                tokens = x.split(char)
                return tokens[0]
    info_dict["product name"] = clean(output_str)

    
    ## find all other info
    trs = container.findAll("tr")
    
    ## loop through info things
    for tr in trs:
        
        # make list, containing key line and value line
        tds = tr.findAll("td")
        
        ## parse out key and value, save to info dict
        key = str(tds[0]).strip("<td>").strip("</")
        value = str(tds[1]).strip("<td>").strip("</").strip().strip("\t")
        # don't care ab last-scanned
        if key != "Last Scanned:":
            info_dict[key]=value
    return info_dict

In [210]:
lookup("013562300587")

{'product name': " Annie's Homegrown Organic Pretzel Bunnies 7 oz Pack of 12",
 'UPC-A:': '0 13562 30058 7',
 'EAN-13:': '0 013562 300587',
 'Amazon ASIN:': 'B00511MLK0',
 'Country of Registration:': 'United States',
 'Brand:': "Annie's, Inc.",
 'Model #:': 'SPK-23102',
 'Weight:': '10 Pounds',
 'Product Dimension:': '9 X 7 X 3 inches'}

#open urllib client, accounting for page not-existing

uClient = uReq("https://www.upcitemdb.com/upc/071641053649")


# grab page html, save as soup
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,"html.parser")

pieces = str(page_soup.head.findAll("meta")[1].meta).split(",")
pieces.pop(0)
print("pieces before splicing:")
print(pieces)

output_str = ""
a=True
for piece in pieces:    
    if piece==" upc lookup database":
        a=False
    print("piece",piece,"a",a)
    if a:
        output_str+=piece
print(output_str)

In [266]:
lookup("013562300112")

{'product name': " Annie's Low Sodium Mac Mild Cheddar Cheese 6 oz",
 'UPC-A:': '0 13562 30011 2',
 'EAN-13:': '0 013562 300112',
 'Amazon ASIN:': 'B0047240A8',
 'Country of Registration:': 'United States',
 'Brand:': "Annie's",
 'Model #:': 'SPK-1390111',
 'Color:': 'Pink',
 'Weight:': '0 pounds',
 'Product Dimension:': '8 X 3 X 5.5 inches'}

In [55]:
import requests

In [212]:
def find_company(brand):

    def side_box_check(brand,page_soup):
        parents = []
        side_box_list = page_soup.findAll("div",{"class":"zloOqf PZPZlf"})
        for item in side_box_list:
            if "Parent" in str(item):
                mini_set = item.findAll("span",{"class":"LrzXr kno-fv"})
                mini_set = str(mini_set[0]).split(",")
                for f1 in mini_set:
                    for snippet in str(f1).split(">"):
                        if "</a" in snippet:
                            parents.append(snippet.strip("</a").replace("&amp;","&"))
        if len(parents)!=0:
            return parents
        else:
            return table_check(brand,page_soup)
        
    def table_check(brand,page_soup):
        try:
            table = page_soup.findAll("div",{"class":"webanswers-webanswers_table__webanswers-table"})[0]
        except:
            return lame_box_check(brand,page_soup)
        rows = table.findAll("tr")
        for tr in rows:
            if "Owner" in str(tr):
                owner_line = str(tr.findAll("td")[1])
                trim1 = owner_line.split(">")[1]
                trim2 = trim1.strip("</td>")
                return trim2.split(",")
    
    def lame_box_check(brand,page_soup):
        output = []
        not_comp = ["company",brand,"parent"]
        try:
            info = page_soup.findAll("span",{"class":"e24Kjd"})[0]
        except IndexError:
            return brand
        bolded = info.findAll("b")
        for bold in bolded:
            bold = str(bold).strip("<b>").strip("</b>")
            if bold not in not_comp:
                output.append(bold)
        for option in output:
            if option in brand or brand in option:
                output.pop(output.index(option))
        return output

    
    my_url = f"https://www.google.com/search?q={brand}+parent+company"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return brand
    
    list_tag = page_soup.findAll("a",{"class":"FLP8od"})
    if len(list_tag)==0:
        list_tag = page_soup.findAll("div",{"class":"Z0LcW AZCkJd"})
        if len(list_tag)==0:
            list_tag = page_soup.findAll("div",{"class":"Z0LcW"})
            if len(list_tag)==0:
                output = []
                intake = side_box_check(brand,page_soup)
                if isinstance(intake,str):
                    return intake
                for item in intake:
                    output.append(item.replace("&amp;","&"))
                return output
    full_tag = str(list_tag[0])
    splitted = full_tag.split(">")
    pre_company = splitted[1]
    post_company = pre_company.split("</")[0]
    return [post_company.replace("&amp;","&")]



In [213]:
print(find_company("Annie's, Inc."))

['General Mills']


In [81]:
def find_ingredients(product):
    import requests
    my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
    query = product.replace(" ","%20")
    response = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={my_api_key}&query={query}")
    big_dict = response.json()
    foods = big_dict["foods"]
    all_matches = []
    for food in foods:
        keys = food.keys()
        if "ingredients" in keys:
            ingred_str = food["ingredients"]
            ingredients = ingred_str.lower().split(",")
            all_matches.append({"name":food["description"],"ingredients":ingredients})
    return all_matches

In [83]:
print(find_ingredients("Naked Berry"))

[{'name': 'NAKED, JUICE SMOOTHIE, BERRY BLAST', 'ingredients': ['apple juice', ' banana puree', ' strawberry puree', ' blackberry puree', ' raspberry puree', ' natural flavors.']}, {'name': 'NAKED, PROTEIN JUICE SMOOTHIE, BERRY', 'ingredients': ['apple juice', ' banana puree', ' strawberry puree', ' soy protein isolate', ' whey protein concentrate', ' blueberry puree', ' soy lecithin', ' fruit and vegetable juice for color', ' natural flavors', ' ascorbic acid (vitamin c).']}, {'name': 'NAKED, 100% JUICE SMOOTHIE, BERRY VEGGIE', 'ingredients': ['sweet cherry puree', ' purple carrot juice from concentrate (water', ' purple carrot juice concentrate)', ' red beet juice from concentrate (water', ' red beet juice concentrate)', ' sweet potato puree', ' strawberry puree', ' plum puree', ' sweet corn puree', ' apple puree', ' chick pea puree', ' lemon juice', ' chicory root fiber', ' natural flavors', ' ascorbic acid (vitamin c)', ' d-alpha tocopheryl acetate (vitamin e)', ' niacinamide', ' d

## TO DOWNLOAD ZIP FILE

response = requests.get(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/GetAllOperationsPublicData?api_key={my_api_key}",stream=True)
target_path = 'organic_ops.zip'
handle = open(target_path, "wb")
for chunk in response.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

import requests
from bs4 import BeautifulSoup as soup

my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
json = {"startldx":"1","count":"1000","countries":["USA"]}
response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Operations?api_key={my_api_key}",json=json)
all_text = response.text

page_soup = soup(all_text,"lxml")
page2 = soup(str(page_soup),"html.parser")
operations_list = page2.html.body.operationsresult.operations.findAll("a:operation")

state_dict = {}
def midinfo(x):
    x = str(x)
    return x.split(">")[1].split("<")[0]
for operation in operations_list:
    operationname = midinfo(operation.findAll("a:operationname")[0])
    operationstatus = midinfo(operation.findAll("a:nopoperationstatus")[0])
    addresses = operation.findAll("a:addresses")
    address = addresses[0].findAll("a:address")
    state_untrimmed = str(address[0].findAll("a:stateorprovince")[0])
    state = midinfo(state_untrimmed)
    print(operationname,operationstatus,state)
    if state in state_dict.keys():
        state_dict[state]+=1
    else:
        state_dict[state]=1
print(state_dict)

all_text = """    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000053</op_nopOpID>
      <op_name>Wong Potatoes, Inc</op_name>
      <op_clientID>OT-017900</op_clientID>
      <op_contFirstName>Daniel</op_contFirstName>
      <op_contLastName>Chin</op_contLastName>
      <op_status>Certified</op_status>
      <op_statusEffectiveDate>2014-11-06</op_statusEffectiveDate>
      <op_nopAnniversaryDate>2020-04-01</op_nopAnniversaryDate>
      <op_lastUpdatedDate>2019-05-15</op_lastUpdatedDate>
      <opSC_HANDLING>Certified</opSC_HANDLING>
      <opSC_HANDLING_ED>2014-11-06</opSC_HANDLING_ED>
      <op_phone>(541) 798-5353</op_phone>
      <op_email>chinfarms@gmail.com</op_email>
      <opMA_line1>17600 Hwy 39</opMA_line1>
      <opMA_city>Klamath Falls</opMA_city>
      <opMA_state>Oregon</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>97603</opMA_zip>
    </Operation>
    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000052</op_nopOpID>
      <op_name>High Mountain LLC</op_name>
      <op_clientID>OT-004440</op_clientID>
      <op_contFirstName>Kevin</op_contFirstName>
      <op_contLastName>Christensen</op_contLastName>
      <op_status>Suspended</op_status>
      <op_statusEffectiveDate>2011-11-16</op_statusEffectiveDate>
      <op_lastUpdatedDate>2017-12-27</op_lastUpdatedDate>
      <opSC_CR>Suspended</opSC_CR>
      <opMA_line1>PO Box 968</opMA_line1>
      <opMA_city>Mattawa</opMA_city>
      <opMA_state>Washington</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>99349</opMA_zip>
    </Operation>
    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000051</op_nopOpID>
      <op_name>Marvin Lynch</op_name>
      <op_otherNames>DBA O'lynch Dairy</op_otherNames>
      <op_clientID>OT-007458</op_clientID>
      <op_contFirstName>Marvin</op_contFirstName>
      <op_contLastName>Lynch</op_contLastName>
      <op_status>Certified</op_status>
      <op_statusEffectiveDate>2009-05-04</op_statusEffectiveDate>
      <op_nopAnniversaryDate>2020-01-01</op_nopAnniversaryDate>
      <op_lastUpdatedDate>2019-05-15</op_lastUpdatedDate>
      <opSC_CR>Certified</opSC_CR>
      <opSC_CR_ED>2009-05-04</opSC_CR_ED>
      <opSC_LS>Certified</opSC_LS>
      <opSC_LS_ED>2009-05-04</opSC_LS_ED>
      <op_phone>(563) 852-5285</op_phone>
      <op_email>marvlynch@yahoo.com</op_email>
      <opMA_line1>24764 Hwy 151 W</opMA_line1>
      <opMA_city>Cascade</opMA_city>
      <opMA_state>Iowa</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>52033</opMA_zip>
    </Operation>

"""


huge_dict={}
all_text = all_text.replace("\n","")
all_operations = all_text.split("</Operation>")
concerns = ["op_nopOpID","op_name","opMA_state"]
for operation in all_operations:
    if "United States" in operation and "status>Certified<" in operation:
        internal_dict = {}
        deets = operation.split("</")
        for deet in deets:
            for concern in concerns:
                if "<"+concern+">" in deet:
                    internal_dict[concern]=deet.split(">")[-1]
        huge_dict[internal_dict["op_name"]] = internal_dict

##CREATING DICTIONARY OF ALL ORGANIC OPERATIONS

all_text = ""
op_file = open("organic_ops_file.txt","r",encoding="utf-8")
for line in op_file.readlines():
    all_text+=line.strip("\n")
op_file.close()

huge_dict={}
all_text = all_text.replace("\n","")
all_operations = all_text.split("</Operation>")
concerns = ["op_nopOpID","op_name","opMA_state"]
for operation in all_operations:
    if "United States" in operation and "status>Certified<" in operation:
        internal_dict = {}
        deets = operation.split("</")
        for deet in deets:
            for concern in concerns:
                if "<"+concern+">" in deet:
                    internal_dict[concern]=deet.split(">")[-1]
        huge_dict[internal_dict["op_name"]] = internal_dict


## WRITING INTO JSON FILE

import json

json_format = json.dumps(mega_dict)
f = open("v2_organic_operations_dict.json","w")
f.write(json_format)
f.close()

In [126]:
#Creating MegaDict
import json

def create_dict(filename="v2_organic_operations_dict.json"):    
    with open(filename,"r") as json_file:
        mega_dict = json.load(json_file)
        if type(mega_dict)=="str":
            print("oops string error")
        else:

            return mega_dict


## trying to fix capitalization in the dictionary
import string
mega_dict = {}
for key in huge_dict.keys():
    if key.isupper():
        fixed = string.capwords(key)
        huge_dict[key]["op_name"] = fixed
        mega_dict[fixed] = huge_dict[key]
    else:
        mega_dict[key] = huge_dict[key]
print(mega_dict)

In [141]:
if not mega_dict:
    mega_dict = create_dict()

def find_operation_id(brand,mega_dict=mega_dict,check_terms=True,nlp = True):
    brand = brand.replace(",","")
    found_keys = []
    dict_keys=mega_dict.keys()
    for key in dict_keys:
        if brand in key:
            found_keys.insert(0,[key,mega_dict[key]["op_nopOpID"]])
    if len(found_keys)==1:
        return found_keys
    elif len(found_keys)!=0 and nlp:
        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize
        
        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(brand) 
        sw = stopwords.words("english") 


        ## NLP filtration method for products
        def check_brand(hit,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 
            Y_list = word_tokenize(hit) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine
        
        for hit in found_keys:
            sim_dict = {}
            name = hit[0]
            similarity_score = check_brand(name)
            sim_dict[similarity_score] = hit
        max_sim = max(sim_dict.keys())
        return sim_dict[max_sim]

    else:
        if check_terms:
            for term in brand.split():
                found_keys.extend(find_operation_id(term,mega_dict,check_terms=False,nlp=nlp))
        found_keys.extend(find_operation_id(find_company(brand),mega_dict,nlp=nlp))
        return found_keys

In [143]:
find_operation_id("Annie's")

[["Annie's Inc.", '8150000355']]

## Receiving Operation Products based on op_code

import requests
from bs4 import BeautifulSoup as soup
#
op_id = "5561001956"
#
my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
json = {"startIdx":"1","count":"1000","operationId":op_id}
response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Items?api_key={my_api_key}",json=json)
# parsing into a list of items
all_text = response.text
page_soup = soup(all_text,"lxml")
page2 = soup(str(page_soup),"html.parser")

items = page2.findAll("a:item")

if len(items)==0:
    return "Error"

In [118]:

## Receiving Operation Products based on op_code

def find_itemvarieties(item):
    deets = item.split("</")
    for deet in deets:
        if "<a:itemvarieties>" in deet:
            itemvarieties = deet.split(">")[-1]
            return itemvarieties

def clean_product(product):
    
    def has_num(string):
        return any(i.isnumeric() for i in string)
    
    out_list = []
    tokens = product.split(",")
    for token in tokens:
        if not has_num(token):
            out_list.append(token)
    return "".join(out_list)
    
        
def organic_info(op_id,product,brand,nlp=True):
    
    import requests
    from bs4 import BeautifulSoup as soup
    if nlp:
        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize 
        
    product = clean_product(product)
    
    do_trash=False
    if do_trash:
        # get rid of common words that are clutter, this list should grow with testing
        trash_list = ["Organic ",brand,"Super ","Green ","Ounce","ounce","count","Count"]
        for stink in trash_list:
            product = product.replace(stink,"").strip()
        for word in product.split():
            for stink in trash_list:
                if stink in word or word.isnumeric():
                    product = product.replace(word,"").strip()
        product = product.split(",")[0]
        product = product.replace(brand,"").strip()
    print("product is",product)
    
    if nlp:

        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(product) 
        sw = stopwords.words("english") 


        ## NLP filtration method for products
        def check_item(itemvarieties,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 

            Y_list = word_tokenize(itemvarieties) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine

    ## API request
    my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
    json = {"startIdx":"1","count":"1000","operationId":op_id}
    response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Items?api_key={my_api_key}",json=json)

    # parsing into a list of items
    all_text = response.text
    page_soup = soup(all_text,"lxml")
    page2 = soup(str(page_soup),"html.parser")
    items = page2.findAll("a:item")

    if len(items)==0:
        return None

    # TEST INFORMATION
    #company = "Forager Project"
    #brand = "Forager Project"
    #product = "Super Green Leafy Greens Chips"
    #

    # create list of API response terms we care about
    concerns = ["itemname","itemvarieties","madewithorganic","organic","organic100"]
    possible_matches = []
    for item in items:
        cont = True
        item = str(item)
        if "status>Certified<" in item:
            try:
                itemvarieties = find_itemvarieties(item)
                if nlp:
                    similarity_score = check_item(itemvarieties)
            except:
                cont=False
            if cont:
                cont2=False
                if nlp and similarity_score>=0.2+0.05*len(possible_matches):
                    internal_dict = {"similarity_score":similarity_score}
                    cont2 = True
                elif not nlp:
                    internal_dict = {"similarity_score":"N/A"}
                    cont2=True
                if cont2:
                    deets = item.split("</")
                    for deet in deets:
                        for concern in concerns:
                            if "<a:"+concern+">" in deet:
                                internal_dict[concern]=deet.split(">")[-1].replace("&amp;","&")
                    possible_matches.append(internal_dict)
    return possible_matches

    # get rid of common words that are clutter, this list should grow with testing
    trash_list = ["Organic ",brand,company,"Super ","Green ","Ounce","ounce","count","Count"]
    for stink in trash_list:
        product = product.replace(stink,"").strip()
    for word in product.split():
        for stink in trash_list:
            if stink in word or word.isnumeric():
                product = product.replace(word,"").strip()

    #print("product before recombo is",product)

    # creating different product variations to test against
    product_combos = [product]    
    for word in product.split():
        product_combos.append(product.replace(word,"").strip())

In [119]:
organic_info("8150000355","Annie's Homegrown Organic Pretzel Bunnies, 7 oz, Pack of 12","Annie's Inc.")

product is Annie's Homegrown Organic Pretzel Bunnies


[{'similarity_score': 0.20412414523193154,
  'itemname': 'Other',
  'itemvarieties': 'Honey Wheat Pretzels Bunnies',
  'organic': 'true'},
 {'similarity_score': 0.2886751345948129,
  'itemname': 'Other',
  'itemvarieties': 'Pretzels Bunnies',
  'organic': 'true'}]

pip install nltk

import nltk


nltk.download("all")

# Program to measure similarity between 
# two sentences using cosine similarity. 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

product = "Super Green Leafy Greens Chips"

# default input to function, only need to tokenize and find stopwords once
X_list = word_tokenize(product) 
sw = stopwords.words('english') 

def compare_terms(itemvarieties,X_list=X_list,sw=sw):
    
    l1 =[];l2 =[] 

    # tokenization 

    Y_list = word_tokenize(itemvarieties) 

    # sw contains the list of stopwords 


    # remove stop words from string 
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector: 
        if w in X_set:
            l1.append(1) # create a vector 
        else:
            l1.append(0) 
        if w in Y_set:
            l2.append(1) 
        else:
            l2.append(0) 

    c = 0

    # cosine formula 
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    return cosine


In [232]:
## testing flow so far

code = input("Please Enter the Barcode of the product you wish to scan:\n")
code_info = lookup(code)
product= code_info["product name"]
brand = code_info["Brand:"]
op_id = find_operation_id(brand)[0][1]
org_inf = organic_info(op_id,product,brand)

Please Enter the Barcode of the product you wish to scan:
013562300587


In [233]:
print(org_inf)

[{'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Pretzels Bunnies', 'organic': 'true'}, {'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Cheddar Bunnies', 'organic': 'true'}, {'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Cheddar Bunnies', 'madewithorganic': 'true'}, {'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Muddy Bunnies', 'organic': 'true'}, {'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Cocoa Bunnies', 'organic': 'true'}, {'similarity_score': 0.31622776601683794, 'itemname': 'Other', 'itemvarieties': 'Berry Bunnies', 'organic': 'true'}]


import requests

product = "Superco Green Cleaner & Degreaser"
safer_choice_url = f"https://enviro.epa.gov/enviro/efservice/T_SAFERCHOICE/PRODUCT NAME/CONTAINING/{product}/JSON"
response = requests.get(safer_choice_url)

dictionary = response.json()

print(dictionary)

# GREEN SEAL by Product 

In [92]:
def find_greenseal(product,nlp=False):

    import requests
    import json
    from bs4 import BeautifulSoup as soup
    if nlp:
        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize

        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(product) 
        sw = stopwords.words("english") 

        ## NLP filtration method for products
        def check_product(product_test,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 

            Y_list = word_tokenize(product_test) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine
    # create search url, define user agent, make request
    search_product = product.replace(" ","%20")
    my_url = f"https://www.greenseal.org/api/search/{search_product}?service_type="
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    # if succesful, employ bs4; return None of get request not success
    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return None
    
    # convert big json response to dictionary and enter results list
    results_dict = json.loads(page_soup.text)
    results_list = results_dict["results"]    
    
    # return None if results is empty
    if len(results_list)==0:
        return None
    
    # initialize empty output list
    item_info = []
    
    # iterate through each result, pulling out info we care about
    for result in results_list:
        mini_dict = {} #empty dictionary to append to list for this product
        product_name = result["name"]
        if nlp and len(item_info)>=1: # if allowing nlp and already found hit, start nlp cross checking
            similarity_score = check_product(product_name)
            if similarity_score >= 0.3+ 0.05*len(item_info):
                mini_dict["name"] = product_name
                mini_dict["company"] = result["mfg"]
                mini_dict["category"] = result["category"]
                item_info.append(mini_dict)
        else:
                mini_dict["name"] = product_name
                mini_dict["company"] = result["mfg"]
                mini_dict["category"] = result["category"]
                item_info.append(mini_dict)

    return item_info 
      
    


results_dict = json.loads(page_soup.text)


item_info = []
results_list = results_dict["results"]
for result in results_list:
    mini_dict = {}
    mini_dict["name"] = result["name"]
    mini_dict["company"] = result["mfg"]
    mini_dict["category"] = result["category"]
    item_info.append(mini_dict)

print(item_info)
    

In [91]:
print(find_greenseal("Tile Cleaner"))

[{'name': 'Surety™ Tub & Tile Cleaner', 'company': 'U S Chemical', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'Tub & Tile Cleaner', 'company': 'Brandless', 'category': 'Household Cleaning Products'}, {'name': 'Surety™ microTECH™ Tub & Tile Cleaner', 'company': 'U S Chemical', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'Surety™ MixMATE™ Tub & Tile Cleaner', 'company': 'U S Chemical', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'Basin Tub & Tile Cleaner', 'company': 'Pak-It', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'EcoPal Tub & Tile Cleaner', 'company': 'Al-Bahar Industries', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'Focus TC 66 Tub & Tile Cleaner', 'company': 'American Cleaning Solutions', 'category': 'Industrial & Institutional Cleaning Products'}, {'name': 'Building Pro TTC Tub & Tile Cleaner', 'company': 'JAD Corporation of America', 'category': 

# NSF International by Company

In [22]:
def find_NSF(company,nlp=False):

    if nlp:
        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize
        
        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(company) 
        sw = stopwords.words("english") 

        ## NLP filtration method for products
        def check_comp(company_name,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 

            Y_list = word_tokenize(company_name) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine


    company = company.replace(" ","%20")
    my_url = f"http://info.nsf.org/Certified/Common/Company.asp?CompanyName={company}&x=0&y=0"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return None
    
    if "No Matching Manufacturers Found" in page_soup.text:
        return None
    
    content_area = page_soup.findAll("div",{"id":"content-area"})[0]
    items = content_area.findAll("table")
    comps_and_cats = []
    for item in items:
        company_name = str(item.td.a.strong.font).split(">")[1].split("</")[0].strip()
        category = str(item.td).split("<br/>")[1].strip("</td>").strip().replace("\r","").replace("\n","").replace("  ","")
        if nlp:
            if len(comps_and_cats)<=1:
                comps_and_cats.append([company_name,category])
            else:
                similarity_score = check_comp(company_name)
                allowable = 0.3+0.1*(len(comps_and_cats))
                if similarity_score >= allowable:
                    omps_and_cats.append([company_name,category])
        elif not nlp:
            comps_and_cats.append([company_name,category])

        
    return comps_and_cats

str

In [24]:
print(find_NSF("Champion"))

[['Champion Chemical Co.', 'NSF Registered Proprietary Substances and Nonfood Compounds'], ['Champion Industries, Inc.', 'NSF/ANSI 2 - Food Equipment']]


In [23]:
print(find_NSF("Champion"))

[['Champion Chemical Co.', 'NSF Registered Proprietary Substances and Nonfood Compounds'], ['Champion Industries, Inc.', 'NSF/ANSI 2 - Food Equipment'], ['Champion Industries, Inc.', 'NSF/ANSI 3 - Commercial Warewashing Equipment'], ['Champion Industries, Inc.', 'NSF/ANSI 13 - Refuse Processors and Processing Systems'], ['Champion Packaging &amp; Distribution, Inc.', 'NSF Registered Proprietary Substances and Nonfood Compounds'], ['Champion Packaging and Distribution Inc.', 'NSF/ANSI/CAN 60 - Drinking Water Treatment Chemicals - Health Effects'], ['Champion-Arrowhead', 'NSF/ANSI/CAN 61 - Drinking Water System Components - Health Effects'], ['Champion-Arrowhead', 'NSF/ANSI/CAN 61 - Drinking Water System Components - Health Effects'], ['Champion-Arrowhead', 'NSF Mechanical Plumbing Products Program'], ['Champion-Arrowhead', 'NSF/ANSI 372 - Drinking Water System Components - Lead Content'], ['ChampionX LLC', 'NSF/ANSI/CAN 60 - Drinking Water Treatment Chemicals - Health Effects'], ['Crash

content_area = page_soup.findAll("div",{"id":"content-area"})[0]
items = content_area.findAll("table")

comps_and_cats = []
for item in items:
    company_name = str(item.td.a.strong.font).split(">")[1].split("</")[0].strip()
    category = str(item.td).split("<br/>")[1].strip("</td>").strip().replace("\r","").replace("\n","").replace("  ","")
    comps_and_cats.append([company_name,category])

## Fair Trade Scraping

In [86]:
def check_fair_trade(company,nlp=False):
    import requests
    from bs4 import BeautifulSoup as soup

    company = company.replace(" ","+")
    fair_trade_url = f"https://www.fairtradecertified.org/search/fair-trade-products?product_type=All&name={company}"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
    company = company.replace("+"," ")

    headers = {"user-agent":user_agent}
    resp = requests.get(fair_trade_url, headers=headers)

    if resp.status_code==200:
        page_soup = soup(resp.content,"html.parser")

    companies = page_soup.findAll("h3")    
    if len(companies)==0:
        return None
    
    if not nlp:
        return [company.text for company in companies]
    else:

        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize

        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(company) 
        sw = stopwords.words("english") 

        ## NLP filtration method for products
        def check_comp(company_name,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 

            Y_list = word_tokenize(company_name) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine


        
        output_list = []
        for company in companies:
            company = company.text
            similarity_score = check_comp(company)
            if similarity_score >= len(output_list)*0.05 + 0.3:
                output_list.append(company)
        return output_list


In [88]:
print(check_fair_trade("Peet's"))

["Peet's Coffee & Tea"]


# ECHO API 

In [45]:
import json
import requests 

def evaluate_facility(mini_dict):
    status = mini_dict["CURR_COMP_STATUS"]
    if "serious" in status:
        return 2
    elif "(s)" in status:
        return 1
    else:
        return 0
    
def violation_score(return_list):
    score=0
    for mini_dict in return_list:
        score+=evaluate_facility(mini_dict)
    return score

def ECHO_violations(brand):
    brand = brand.upper()
    print(brand)
    api_url = f"https://enviro.epa.gov/enviro/efservice/t_compliance_echo/NAME/CONTAINING/{brand}/JSON"
    response = requests.get(api_url)
    if response.status_code == 200:
        return_list = json.loads(response.text)
        if len(return_list)==0:
            return None
        else:
            vio_ratio = violation_score(return_list)/len(return_list)
            return vio_ratio*5  
    else:
        return None



In [84]:

print(ECHO_violations("Forager"))


FORAGER
None


## Non-GMO Project Check 
still need to rewrite so that will check brand, not product

In [67]:
def nonGMO(product,brand,nlp=False,first_try=True):
    
    import json
    
    combo = brand+" "+product
    combo = combo.strip()
    api_url = f"https://ws2.nongmoproject.org/api/v1/get_brands_products_by_keyword?keyword={combo}&page=1"
    response = requests.get(api_url)
    if response.status_code==200:
        results_dict = json.loads(response.text)
        results_list = results_dict["data"]
    else:
        return None
        


    if nlp:

        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize

        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(product) 
        sw = stopwords.words("english") 

        ## NLP filtration method for products
        def check_comp(company_name,X_list=X_list,sw=sw):

            l1 =[];l2 =[] 

            # tokenization 

            Y_list = word_tokenize(company_name) 

            # sw contains the list of stopwords 


            # remove stop words from string 
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw} 

            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector: 
                if w in X_set:
                    l1.append(1) # create a vector 
                else:
                    l1.append(0) 
                if w in Y_set:
                    l2.append(1) 
                else:
                    l2.append(0) 

            c = 0

            # cosine formula 
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine


    matches = []


    for match in results_list:
        name = match["name"]
        if len(matches)==0 or not nlp:
            matches.append({"name":name,"total_products":match["total_products"]})
        else:
            similarity_score = check_comp(name)
            if similarity_score >= 0.3 + 0.05*len(matches):
                matches.append({"name":name,"total_products":match["total_products"]})

    if len(matches)>0:
        return matches
    elif first_try:
        return nonGMO(brand,"",nlp,first_try=False)
    else:
        return None

In [57]:
print(matches)

[{'name': "Annie's", 'total_products': 114}, {'name': "Annie's Bakery", 'total_products': 10}, {'name': "Annie's Best", 'total_products': 3}]


In [68]:
print(nonGMO("bunnies","Annie's"))

[{'name': "Annie's", 'total_products': 114}, {'name': "Annie's Bakery", 'total_products': 10}, {'name': "Annie's Best", 'total_products': 3}]


## CDP A-List

In [29]:
def CDP_A(company,nlp=False):
    import json
    with open("cdp_dict.json","r") as json_file:
        cdp_dict = json.loads(json_file.readline())
    
    possible = []
    company = company.lower()
    
    for key in cdp_dict.keys():
        if company in key.lower() or key.lower() in company:
            possible.append([key,cdp_dict[key]])
    if len(possible)==0:
        return None
    elif len(possible)==1:
        return possible[0][1]
    else:
        if not nlp:
            added = 0
            for option in possible:
                added+=option[1]
            return added/len(possible)
        else:
            from nltk.corpus import stopwords 
            from nltk.tokenize import word_tokenize

            # default input to similarity function, only need to tokenize and find stopwords once
            X_list = word_tokenize(company) 
            sw = stopwords.words("english") 

            ## NLP filtration method for products
            def check_comp(company_name,X_list=X_list,sw=sw):

                l1 =[];l2 =[] 

                # tokenization 

                Y_list = word_tokenize(company_name) 

                # sw contains the list of stopwords 


                # remove stop words from string 
                X_set = {w for w in X_list if not w in sw} 
                Y_set = {w for w in Y_list if not w in sw} 

                # form a set containing keywords of both strings 
                rvector = X_set.union(Y_set) 
                for w in rvector: 
                    if w in X_set:
                        l1.append(1) # create a vector 
                    else:
                        l1.append(0) 
                    if w in Y_set:
                        l2.append(1) 
                    else:
                        l2.append(0) 

                c = 0

                # cosine formula 
                for i in range(len(rvector)): 
                    c+= l1[i]*l2[i] 
                cosine = c / float((sum(l1)*sum(l2))**0.5)
                return cosine
            
            round2_dict = {}
            for option in possible:
                sim_score = check_comp(option[0])
                round2_dict[sim_score] = option
            best_list = round2_dict[max(round2_dict.keys())]
            return best_list[1]

In [31]:
print(CDP_A("Deutsche",nlp=False))

1.0


# Rainforest Alliance

In [90]:
def ra_check(brand):
    import requests
    from bs4 import BeautifulSoup as soup

    ra_url = f"https://www.rainforest-alliance.org/find-certified?location=330&category=&keyword={brand}&op=submit"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(ra_url, headers=headers)

    if resp.status_code!=200:
        return None
    else:
        page_soup = soup(resp.text,"html.parser")
        
    companies = page_soup.findAll("div",{"class":"cp-teaser-info-content"})
    
    useful_dict = {}
    for company in companies:
        comp_name = company.h4.span.text
        products = company.div.findAll("span")
        cleaned_products = []
        for product in products:
            if products.index(product)!=0:
                cleaned_products.append(product.text.strip().strip(","))
        useful_dict[comp_name] = cleaned_products
        
        
    return useful_dict

In [91]:
print(ra_check("Peet's"))

{"Peet's Coffee & Tea": ['Coffee']}



Coffee

Coffee
